In [1]:
import numpy as np
from bokeh.io import push_notebook, output_notebook
from bokeh.plotting import figure, show
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os
import pickle
import imageio

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
CABLE_DELAYS = 0.65 * 2
FOLDER_NAME = '2019-01-30 20-03-18'

In [4]:
with open(os.path.join(FOLDER_NAME, 'data.pickle'), 'rb') as f:
    frequencies = pickle.load(f)
    sweeps = pickle.load(f)
    frame_time = pickle.load(f)
sweeps = [tmp[0][1]['data'] for tmp in sweeps]

In [5]:
step = abs(frequencies[0] - frequencies[-1]) / len(frequencies)
front_padding_count = max(int(frequencies[0] / step), 0)

In [6]:
time_domain_data = []
for i in range(len(sweeps)):
    data_pt = sweeps[i]

    padded_data = []

    while len(padded_data) < front_padding_count:
        padded_data.append(0)
    padded_data.extend(data_pt)

    powers_of_two = [2 ** x for x in range(16)]

    for size in powers_of_two:
        if (size > len(padded_data)):
            final_size = size
            break
    
    while len(padded_data) < final_size:
        padded_data.append(0)
    
    padded_data = np.array(padded_data)

    time_domain_data.append(np.fft.ifft(padded_data))
    
time_domain_data = np.abs(time_domain_data[:len(time_domain_data) // 2])
axis = np.array(range(time_domain_data.shape[1]))

step = step * 1e6 # Hertz
axis = axis * (1 / (len(axis) * step * 2)) # Hertz to seconds
axis = axis * 1e9 # Nanoseconds
axis = axis - CABLE_DELAYS
axis = axis * 0.983571 # Nanoseconds to feet

In [17]:
p = figure(plot_width=800, plot_height=600, y_axis_type='log', y_range=(10 ** -2, 15e2))
l = p.line(axis, time_domain_data[0], line_width=1)

In [18]:
def update(sweep):
    l.data_source.data['y'] = time_domain_data[sweep - 1]
    push_notebook()

In [19]:
show(p, notebook_handle=True);

In [20]:
interact(update, sweep=(1,50,1));

interactive(children=(IntSlider(value=25, description='sweep', max=50, min=1), Output()), _dom_classes=('widge…